In [1]:
import pandas as pd
import numpy as np

In [2]:
school = pd.read_csv('../data/school.csv',encoding_errors='ignore')
subway = pd.read_csv('../data/subway_feature.csv')
bus = pd.read_csv('../data/bus_feature.csv')
address = pd.read_csv('../data/addresses.csv')

In [3]:
address.head()

,주소,경도,위도
0,서울특별시 강남구 개포동 언주로 3,127.056841,37.476283
1,서울특별시 강남구 개포동 개포로 307,127.056014,37.483973
2,서울특별시 강남구 개포동 개포로109길 69,127.076626,37.496296
3,서울특별시 강남구 개포동 개포로 310,127.064582,37.486862
4,서울특별시 강남구 개포동 선릉로 7,127.062627,37.480291


In [4]:
school = school.rename(columns={'위도':'Y', '경도':'X'})
subway = subway.rename(columns={'위도':'Y', '경도':'X'})
bus = bus.rename(columns={'Y좌표':'Y', 'X좌표':'X'})
address = address.rename(columns={'위도':'Y', '경도':'X'})

In [5]:
def add_poi_counts(address_df: pd.DataFrame,
                   poi_df: pd.DataFrame,
                   radii=(1, 3, 5, 10),
                   lat_col='위도',
                   lon_col='경도',
                   suffix='poi') -> pd.DataFrame:
    """
    주소 데이터프레임(address_df)과 POI 데이터프레임(poi_df)을 받아,
    각 반경(radii) 내 POI 개수를 주소 행별로 계산해
    f'{r}_{suffix}' 이름의 컬럼으로 추가합니다.

    - 좌표는 km 단위 하버사인 거리로 계산
    - 주소/POI의 위경도 결측은 자동 제외
    """
    # 유효 좌표만 남기기
    poi_valid = poi_df.dropna(subset=[lat_col, lon_col]).copy()
    addr_valid_mask = address_df[lat_col].notna() & address_df[lon_col].notna()

    if poi_valid.empty or not addr_valid_mask.any():
        # 결과 컬럼만 NaN으로 생성
        for r in radii:
            address_df[f'{r}_{suffix}'] = np.nan
        return address_df

    # 넘파이 배열로 변환 (라디안)
    addr_lat = np.deg2rad(address_df.loc[addr_valid_mask, lat_col].to_numpy())
    addr_lon = np.deg2rad(address_df.loc[addr_valid_mask, lon_col].to_numpy())
    poi_lat = np.deg2rad(poi_valid[lat_col].to_numpy())
    poi_lon = np.deg2rad(poi_valid[lon_col].to_numpy())

    # 하버사인 거리 행렬 계산 (단위: km)
    R = 6371.0088  # 지구 반경(km)
    dlat = addr_lat[:, None] - poi_lat[None, :]
    dlon = addr_lon[:, None] - poi_lon[None, :]
    a = np.sin(dlat / 2) ** 2 + np.cos(addr_lat)[:, None] * np.cos(poi_lat)[None, :] * np.sin(dlon / 2) ** 2
    dist = 2 * R * np.arcsin(np.sqrt(a))

    # 반경별 개수 계산 및 컬럼 채우기
    for r in radii:
        col = f'{r}_{suffix}'
        counts = (dist <= r).sum(axis=1).astype(int)
        address_df[col] = np.nan
        address_df.loc[addr_valid_mask, col] = counts

    return address_df

In [6]:
# 주소별 버스 수 컬럼 추가
address = add_poi_counts(
    address_df=address,
    poi_df=bus,
    radii=(1, 3, 5, 10),
    lat_col='Y',
    lon_col='X',
    suffix='bus'
)

# 결과 확인
address.head()

/tmp/ipykernel_510326/1139459010.py:43: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  address_df.loc[addr_valid_mask, col] = counts
/tmp/ipykernel_510326/1139459010.py:43: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  address_df.loc[addr_valid_mask, col] = counts
/tmp/ipykernel_510326/1139459010.py:43: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-uniqu

,주소,X,Y,1_bus,3_bus,5_bus,10_bus
0,서울특별시 강남구 개포동 언주로 3,127.056841,37.476283,57,439,1059,3424
1,서울특별시 강남구 개포동 개포로 307,127.056014,37.483973,77,500,1135,3771
2,서울특별시 강남구 개포동 개포로109길 69,127.076626,37.496296,49,397,1221,3698
3,서울특별시 강남구 개포동 개포로 310,127.064582,37.486862,62,434,1136,3651
4,서울특별시 강남구 개포동 선릉로 7,127.062627,37.480291,52,430,1059,3444


In [7]:
# 주소별 지하철 수 컬럼 추가
address = add_poi_counts(
    address_df=address,
    poi_df=subway,
    radii=(1, 3, 5, 10),
    lat_col='Y',
    lon_col='X',
    suffix='sub'
)

# 결과 확인
address.head()

/tmp/ipykernel_510326/1139459010.py:43: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  address_df.loc[addr_valid_mask, col] = counts
/tmp/ipykernel_510326/1139459010.py:43: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  address_df.loc[addr_valid_mask, col] = counts
/tmp/ipykernel_510326/1139459010.py:43: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-uniqu

,주소,X,Y,1_bus,3_bus,5_bus,10_bus,1_sub,3_sub,5_sub,10_sub
0,서울특별시 강남구 개포동 언주로 3,127.056841,37.476283,57,439,1059,3424,0,14,46,156
1,서울특별시 강남구 개포동 개포로 307,127.056014,37.483973,77,500,1135,3771,4,20,50,174
2,서울특별시 강남구 개포동 개포로109길 69,127.076626,37.496296,49,397,1221,3698,3,26,61,173
3,서울특별시 강남구 개포동 개포로 310,127.064582,37.486862,62,434,1136,3651,6,21,55,170
4,서울특별시 강남구 개포동 선릉로 7,127.062627,37.480291,52,430,1059,3444,1,16,50,164


In [8]:
# 주소별 초등학교 수 컬럼 추가
address = add_poi_counts(
    address_df=address,
    poi_df=school,
    radii=(1, 3, 5, 10),
    lat_col='Y',
    lon_col='X',
    suffix='school'
)

# 결과 확인
address.head()

/tmp/ipykernel_510326/1139459010.py:43: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  address_df.loc[addr_valid_mask, col] = counts
/tmp/ipykernel_510326/1139459010.py:43: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  address_df.loc[addr_valid_mask, col] = counts
/tmp/ipykernel_510326/1139459010.py:43: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-uniqu

,주소,X,Y,1_bus,3_bus,5_bus,10_bus,1_sub,3_sub,5_sub,10_sub,1_school,3_school,5_school,10_school
0,서울특별시 강남구 개포동 언주로 3,127.056841,37.476283,57,439,1059,3424,0,14,46,156,4,24,52,172
1,서울특별시 강남구 개포동 개포로 307,127.056014,37.483973,77,500,1135,3771,4,20,50,174,7,25,58,198
2,서울특별시 강남구 개포동 개포로109길 69,127.076626,37.496296,49,397,1221,3698,3,26,61,173,4,31,73,197
3,서울특별시 강남구 개포동 개포로 310,127.064582,37.486862,62,434,1136,3651,6,21,55,170,7,24,60,192
4,서울특별시 강남구 개포동 선릉로 7,127.062627,37.480291,52,430,1059,3444,1,16,50,164,5,25,56,182


In [9]:
# 접근성 점수(가까운 반경에 더 큰 가중치)
def _weighted_score(df, cols_weights):
    score = np.zeros(len(df))
    for col, w in cols_weights:
        if col in df.columns:
            score += w * pd.to_numeric(df[col], errors="coerce").fillna(0).to_numpy()
    return score

bus_weights = [("1_bus", 1.0), ("3_bus", 0.6), ("5_bus", 0.3), ("10_bus", 0.1)]
sub_weights = [("1_sub", 1.0), ("3_sub", 0.6), ("5_sub", 0.3), ("10_sub", 0.1)]
sch_weights = [("1_school", 1.0), ("3_school", 0.6), ("5_school", 0.3), ("10_school", 0.1)]

address["bus_access_score"] = _weighted_score(address, bus_weights)
address["sub_access_score"] = _weighted_score(address, sub_weights)
address["school_access_score"] = _weighted_score(address, sch_weights)

In [10]:
address.head()

,주소,X,Y,1_bus,3_bus,5_bus,10_bus,1_sub,3_sub,5_sub,10_sub,1_school,3_school,5_school,10_school,bus_access_score,sub_access_score,school_access_score
0,서울특별시 강남구 개포동 언주로 3,127.056841,37.476283,57,439,1059,3424,0,14,46,156,4,24,52,172,980.5,37.8,51.2
1,서울특별시 강남구 개포동 개포로 307,127.056014,37.483973,77,500,1135,3771,4,20,50,174,7,25,58,198,1094.6,48.4,59.2
2,서울특별시 강남구 개포동 개포로109길 69,127.076626,37.496296,49,397,1221,3698,3,26,61,173,4,31,73,197,1023.3,54.2,64.2
3,서울특별시 강남구 개포동 개포로 310,127.064582,37.486862,62,434,1136,3651,6,21,55,170,7,24,60,192,1028.3,52.1,58.6
4,서울특별시 강남구 개포동 선릉로 7,127.062627,37.480291,52,430,1059,3444,1,16,50,164,5,25,56,182,972.1,42.0,55.0


In [11]:
address.to_csv('address_with_poi.csv', index=False)